In [3]:
pip install requests_html

     |████████████████████████████████| 83 kB 1.2 MB/s 
     |████████████████████████████████| 138 kB 13.1 MB/s 
     |████████████████████████████████| 111 kB 42.6 MB/s 
     |████████████████████████████████| 127 kB 39.5 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=c7f4c1d1d44cbdfe39e90feaeea43c494b3549628251ba98078feb6cafdf69c6
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=85f604ecd4acc9c5ee9df5fcc683bd9a74de6788ccfdbf8599563f7ec9bcfc2d
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
Successfully built fake-useragent parse
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not

In [10]:
import time
import requests
import pandas as pd
import numpy as np
from urllib.error import HTTPError
from requests_html import HTMLSession


In [4]:
def get_cik(session, ticker):
    print(f'Getting CIK code for {ticker}')

    cik_code = ""

    try:
        url = f'https://sec.report/Ticker/{ticker}'
        r = session.get(url, timeout=30)

        cik_text = r.html.xpath('/html/body/div[1]/div/h2[1]')[0].text
        cik_code = cik_text.split(' ')[2]
        cik_code_numbers = filter(str.isdigit, cik_code)
        cik_code = "".join(cik_code_numbers)

        print(f'{ticker} CIK: {cik_code}')
    except HTTPError as err:
        print(f'Requesting {ticker}: {err}')
    except:
        print(f'Couldn\'t get CIK for {ticker}')
    
    return cik_code

In [5]:
class FormsNotFoundException(Exception):
    """Raise when neither S-1 nor F-1 form is found"""


In [6]:
def get_submissions(cik_code):
    print(f'Getting submissions for {cik_code}')
    url = f'https://data.sec.gov/submissions/CIK{cik_code}.json'
    subs = ''
    
    if cik_code != '':
        try:
            headers = {
                'User-Agent': 'Mike Brover (mike.brover@gmail.com)',
                'From': 'john.testorsu@gmail.com'
            }
            res = requests.get(url, headers=headers, timeout=20)
            subs = res.json()
        except:
            print(f'Failed to get submissions for {cik_code}')

    return subs

In [7]:
def get_access_codes(submissions):
    accession_number = ''
    primary_document = ''

    if submissions != '':
        try:
            recent_filings = submissions['filings']['recent']
            index = -1
            for i, form_name in enumerate(recent_filings['form']):
                if ('S-1' in form_name) or ('F-1' in form_name):
                    index = i
                    break

            if index == -1:
                print('Neither S-1 nor F-1 form was found')
                return accession_number, primary_document

            accession_number = recent_filings['accessionNumber'][index].replace('-', '')
            primary_document = recent_filings['primaryDocument'][index]
        except:
            print('Failed to get access codes')

    return accession_number, primary_document

In [8]:
def get_report_url(cik_code, accession_number, primary_document):
    if cik_code == '' or accession_number == '' or primary_document == '':
        return 'Not found'
    return f'https://www.sec.gov/Archives/edgar/data/{cik_code}/{accession_number}/{primary_document}'

In [12]:
!gdown --id 1dY2urwIk-1sdwFdXz8fZymxtfGqS0JeC

Downloading...
From: https://drive.google.com/uc?id=1dY2urwIk-1sdwFdXz8fZymxtfGqS0JeC
To: /content/data_features_001 (3).zip
100% 57.1k/57.1k [00:00<00:00, 44.1MB/s]


In [13]:
from zipfile import ZipFile
file_name = '/content/data_features_001 (3).zip'

with ZipFile(file_name, 'r') as zipi:
  zipi.extractall()
  print('Done')

Done


In [16]:
df = pd.read_csv('/content/data_features_001 (3).csv', index_col=None, na_values=['NA'])

In [ ]:
tickers = df['Symbol'].to_numpy() 

print(tickers)

session = HTMLSession()
report_urls = {}
for i, ticker in enumerate(tickers):
    try:
        print(f'{i}/{len(tickers)} - {ticker}')
        cik_code = get_cik(session, ticker)
        sub_json = get_submissions(cik_code)
        accession_number, primary_document = get_access_codes(sub_json)
        report_url = get_report_url(cik_code, accession_number, primary_document)

        report_urls[ticker] = [report_url]
        print('------------------')
        time.sleep(0.2)
    except:
        print(f'Failed to get url of {ticker}')

out_df = pd.DataFrame.from_dict(report_urls, orient='index', columns=['URL'])
print(out_df)
out_df.to_csv('/content/drive/My Drive/IPO_Project/data/processed_data/report_urls.csv') 